In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import re
import requests
import datetime
from datetime import datetime
from bs4 import BeautifulSoup
import bs4.element
from tqdm.notebook import tqdm
from konlpy.tag import Komoran

from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

C:\Users\mtang\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Crawling 

In [17]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'
today = str(datetime.now().strftime('%Y%m%d'))
print(today)

def get_soup_obj(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            news_contents += content.strip() + ' '

    return news_contents

def get_news_info(url, s) : 
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    current_page = 1     
    news_info_list = []

    for i in range (10) : 
        sec_url = url + s + "&date=" + today + "&page=" + str(current_page)
        soup = get_soup_obj(sec_url)
        lis = soup.find('ul', class_='type06_headline').find_all("li", limit=15)
        
        for li in lis : 
            news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img,
            "category" : s }
            try :
                news_contents = get_news_contents(news_info['news_url'])
                news_info['contetns'] = news_contents
                news_info_list.append(news_info)
            except Exception as e : 
                continue
        
        current_page += 1 
    
    print(s + " 분야 크롤링 완료")    
    return news_info_list

sid = ['100', '101', '102', '103', '104', '105']
default_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1="
df = pd.DataFrame()

for s in tqdm(sid) : 
    news = get_news_info(default_url, s)
    df = df.append(news)

df.to_csv('네이버뉴스_{}.csv'.format(date), encoding = "utf-8-sig")

20211115


  0%|          | 0/6 [00:00<?, ?it/s]

100 분야 크롤링 완료
101 분야 크롤링 완료
102 분야 크롤링 완료
103 분야 크롤링 완료
104 분야 크롤링 완료
105 분야 크롤링 완료


# Data Loading